# Zipline with pandas panel, LKTB, krx calendar

In [ ]:
from zipline.api import order, record, symbol, set_benchmark
import zipline
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from collections import OrderedDict
import pytz

## OrderedDict 이용해서 pandas panel 만들기

In [ ]:
full_file_path = "./data/SPY.csv"
data = OrderedDict()
data['SPY'] = pd.read_csv(full_file_path, index_col=0, parse_dates=['date'])
data['SPY'] = data['SPY'][["open","high","low","close","volume"]]
print(data['SPY'].head())

In [ ]:
panel = pd.Panel(data)
panel.minor_axis = ['open','high','low','close','volume']
panel.major_axis = panel.major_axis.tz_localize(pytz.utc) # 2018-01-02 -> 2018-01-02 00:00:00+00:00
print(panel)

In [ ]:
type(panel)

## 벤치마크 설정

In [ ]:
def initialize(context):
    set_benchmark(symbol("SPY"))

## handle_data 세팅

In [ ]:
def handle_data(context, data):
    order(symbol("SPY"), 10)
    record(SPY=data.current(symbol('SPY'), 'price'))

## 백테스팅 실행 (data에 pandas panel을 넣어준다)

In [ ]:
# 날짜 유의
perf = zipline.run_algorithm(start=datetime(2017, 1, 5, 0, 0, 0, 0, pytz.utc),
                      end=datetime(2018, 3, 1, 0, 0, 0, 0, pytz.utc),
                      initialize=initialize,
                      capital_base=100000,  # 기초자산 설정
                      handle_data=handle_data,
                      data=panel)

## 결과 비교

테스팅 결과와 벤치마크인 SPY 지수의 수익률 비교를 위해 SPY 일일 수익률을 생성한다.

In [ ]:
data = OrderedDict()
data['SPY'] = pd.read_csv(full_file_path, index_col=0, parse_dates=['date'])
data['SPY'] = data['SPY'][["open","high","low","close","volume"]]
data['SPY'] = data['SPY'].resample("1d").mean()
data['SPY'].fillna(method="ffill", inplace=True)
print(data['SPY'].head())

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style

style.use("ggplot")

perf.portfolio_value.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='portfolio')
perf.SPY.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='benchmark')
plt.legend(loc=2)

plt.show()

In [ ]:
perf.max_leverage.plot()
plt.show()

In [ ]:
data = data['CLOSE']
data.columns = ['LKTB']
data = data.tz_localize('UTC')
data.tail()

In [ ]:
start = datetime.datetime(2012,1,1)
end = datetime.datetime(2019,12,31)

data = data[start:end]

In [ ]:
data

In [ ]:
from zipline.api import order_target, record, symbol, set_commission, commission, set_slippage, slippage

In [ ]:
import zipline

In [ ]:
def initialize(context):
    context.i = 0
    context.sym = symbol('LKTB')
    context.hold = False
    set_commission(commission.PerShare(cost=0.000008))
    set_slippage(slippage.FixedSlippage(spread=0))

In [ ]:
def handle_data(context, data):
    context.i += 1
    if context.i < 20:
        return
    
    buy = False
    sell = False
    
    ma5 = data.history(context.sym, 'price', 5, '1d').mean()
    ma20 = data.history(context.sym, 'price', 20, '1d').mean()
    
    if ma5 > ma20 and context.hold == False:
        order_target(context.sym, 100)
        context.hold = True
        buy = True
    elif ma5 < ma20 and context.hold == True:
        order_target(context.sym, -100)
        context.hold = False
        sell = True
        
    record(LKTB=data.current(context.sym, "price"), ma5=ma5, ma20=ma20, buy=buy, sell=sell)

In [ ]:
# from zipline.algorithm import TradingAlgorithm

In [ ]:
from trading_calendars import get_calendar
trading_calendar=get_calendar('XKRX')

In [ ]:
from zipline import run_algorithm

start = datetime.datetime(2012, 1, 2)
end = datetime.datetime(2019,12,30)

start_utc = start.replace(tzinfo = datetime.timezone.utc)
end_utc = end.replace(tzinfo = datetime.timezone.utc)

result = run_algorithm(
    start = start_utc, 
    end = end_utc, 
    initialize = initialize, 
    capital_base = 1000, 
    handle_data = handle_data, 
    trading_calendar=trading_calendar,
    data = data
)

In [ ]:
start_utc

In [ ]:
'''
from zipline.utils.factory import create_simulation_parameters
algo = TradingAlgorithm(initialize=initialize, handle_data=handle_data, trading_calendar=trading_calendar)
# algo = TradingAlgorithm(initialize=initialize, handle_data=handle_data)
result = algo.run(data)
'''

In [ ]:
plt.plot(result.index, result.portfolio_value)
plt.show()

In [ ]:
result.info()

In [ ]:
plt.plot(result.index, result.LKTB)
plt.plot(result.index, result.ma5)
plt.plot(result.index, result.ma20)

plt.legend(loc='best')

plt.plot(result[result.buy == True].index, result.LKTB[result.buy == True], '^')
plt.plot(result[result.sell == True].index, result.LKTB[result.sell == True], 'v')
plt.show()

In [ ]:
result[['starting_cash', 'ending_cash', 'ending_value', 'portfolio_value', 'LKTB', 'capital_used']].head(50)

In [ ]:
result.tail()

In [ ]:
import pyfolio as pf


In [ ]:
returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(result)

In [ ]:
pf.plot_drawdown_periods(returns, top=5).set_xlabel('Date')

In [ ]:
transactions

In [ ]:
#pf.create_simple_tear_sheet(returns, positions=positions, transactions=transactions, live_start_date='2019-02-10')
#pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, live_start_date='2019-02-10', round_trips=True)
#pf.create_simple_tear_sheet(returns, positions=positions, transactions=transactions,live_start_date='2019-02-10')
sheets = pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions)

In [ ]:
sheets

In [ ]:

returns.shape

In [ ]:
returns.head()

In [ ]:
type(returns)

In [ ]:
returns.index = returns.index.date

In [ ]:
returns

In [ ]:
#pf.create_simple_tear_sheet(returns, positions=positions, transactions=transactions, live_start_date='2019-02-10')
pf.create_simple_tear_sheet(returns)
